In [ ]:
## Create Sub Folders

In [ ]:
image_types = ['.jpg', '.png', '.tif']
folders = [
           fname for fname in zipfile.namelist() if 
           #not any(ftype for ftype in image_types if ftype in fname)
           '.' not in fname
           ]
replacing = {
    'ds/images/remote_sensing/otherDatasets/sentinel_2':'',
    'tif':'MS'
}
#replacing = {
#    '2750':'RGB'
#}

def replace_list_dict(flist, sdict):
  for good,bad in sdict.items():
    flist = [f.replace(good,bad).strip('/') for f in flist]
  return flist

folders = replace_list_dict(folders, replacing)

os.chdir(PROJ_DIR)
print(os.getcwd())

def create_path(NEW_DIR, BASE_DIR=PROJ_DIR):
  os.chdir(BASE_DIR)
  if os.path.exists(NEW_DIR) == False:
    os.mkdir(NEW_DIR)

for folder in folders:
  create_path(folder) 
  print(folder)

## Prepare Files as Dict

In [ ]:
from tqdm import tqdm

def split_path_name(full_filename):
  filepath, filename = full_filename.rsplit('/', 1)
  return filepath, filename

replacing = {
    'ds/images/remote_sensing/otherDatasets/sentinel_2':'',
    'tif/':'MS/'
}
#replacing = {
#    '2750/':'RGB/'
#}


def replace_dict(rstr, sdict):
  for good,bad in sdict.items():
    rstr = rstr.strip('/').replace(good,bad)
  return rstr

def replace_list_dict(flist, sdict):
  for good,bad in sdict.items():
    flist = [f.strip('/').replace(good,bad) for f in flist]
  return flist



zip_info = zipfile.infolist()
#zip_info = zip_info[500:505]

zip_names = {}
for zfile in zip_info:
    full_filename = zfile.filename

    if '.' in full_filename:
        
      full_filename = replace_dict(full_filename, replacing)

      parts = full_filename.split('/')
      if '_' in parts[0]:
        new_sub = parts[0].split('_')[0]
        full_filename = new_sub + '/' + full_filename
        print(full_filename)
      fpath, fname = split_path_name(full_filename)
      if os.path.isfile(os.path.join(fpath,fname)) == False:
        if fpath not in zip_names.keys():
          zip_names[fpath] = []
        zfile.filename = fname
        zip_names[fpath].append({
              'name': fname,
              'path': fpath,
              'file': zfile
              })

## Upload Files by Subfolder

In [ ]:

os.chdir(PROJ_DIR)
%cd {PROJ_DIR}
errors = []
tried=[]

for dirname, filelist in zip_names.items():
  UPLOAD_DIR = os.path.join(PROJ_DIR, dirname)
  os.chdir(UPLOAD_DIR)
  %cd {UPLOAD_DIR}
  for fdict in tqdm(filelist, desc='{} '.format(dirname)):
    fname, fpath, zfile = fdict['name'], fdict['path'], fdict['file']
    try:
      
      zipfile.extract(zfile)
      #tried.append(UPLOAD_DIR+'/'+fname)
    except:
      errors.append(fname, UPLOAD_DIR)

In [ ]:
tried

## UPLOAD TO GOOGLE DRIVE

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# To create a new folder
folder_name = input("Name the Folder : ")
folder = drive.CreateFile({'title' : folder_name, 'mimeType' : 'application/vnd.google-apps.folder'})
folder.Upload()